In [161]:
import numpy as np
import pandas as pd
import xgboost
import yaml
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import shap
from sklearn.model_selection import StratifiedKFold

class generate_model():
    
    def __init__(self, predictor_col: str, data_path: str,xg_params: dict, kfold_splits : int , test_size=0.3, removed_features: list[str] = None, seed: float = None) -> None:
        if removed_features is None:
            removed_features = []
        dataset = pd.read_csv(data_path)

        # Variable encoding
        dataset["Género"] = dataset["Género"].replace({"M": 0, "F": 1}).astype("category")
        dataset["ATPII/AHA/IDF"] = (
            dataset["ATPII/AHA/IDF"].replace({"no": 0, "si": 1}).astype("category")
        )
        dataset["aleator"] = (
            dataset["aleator"]
            .replace({"Control": 0, "PKU 1": 1, "PKU 2": 2})
            .astype("category")
        )

        y_df = dataset[predictor_col].replace({"No": 0, "Si": 1}).astype("category")
        X_df = dataset.drop(predictor_col, axis="columns")

        X_df = X_df.drop(removed_features, axis="columns")

        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
                                                                X_df, y_df, stratify=y_df, test_size=test_size, random_state=seed
                                                            )
    #def xg_train(self, xg_params: dict, kfold_splits=5, seed=None):

        cv = StratifiedKFold(n_splits=kfold_splits, shuffle=True, random_state=seed)
        folds = list(cv.split(self.X_train, self.y_train))

        for train_idx, val_idx in folds:
            # Sub-empaquetado del train-set en formato de XGBoost
            dtrain = xgboost.DMatrix(
                self.X_train.iloc[train_idx, :],
                label=self.y_train.iloc[train_idx],
                enable_categorical=True,
            )
            dval = xgboost.DMatrix(
                self.X_train.iloc[val_idx, :],
                label=self.y_train.iloc[val_idx],
                enable_categorical=True,
            )

            self.model = xgboost.train(
                dtrain=dtrain,
                params=xg_params,
                evals=[(dtrain, "train"), (dval, "val")],
                num_boost_round=1000,
                verbose_eval=False,
                early_stopping_rounds=10,
            )
    def get_AUC_on_test_data(self):
        #def xg_test(model, X_test, y_test) -> float:
        testset = xgboost.DMatrix(self.X_test, label=self.y_test, enable_categorical=True)
        y_preds = self.model.predict(testset)

        return roc_auc_score(testset.get_label(), y_preds)
    
    
    def get_feature_metrics(self):
        # internal_feature_metrics = pd.DataFrame(
        #     {
        #         "Weight": self.model.get_score(importance_type="weight"),
        #         "Coverage": self.model.get_score(importance_type="cover"),
        #         "Gain": self.model.get_score(importance_type="gain"),
        #     }
        # ).sort_values(by="Gain", ascending=False)

        explainer = shap.TreeExplainer(self.model)

        # Extrae la explicacion SHAP en un DF
        EXPLAINATION = explainer(self.X_test).cohorts(
            self.y_test.replace({0: "Healty", 1: "Abnormal"}).tolist()
        )
        cohort_exps = list(EXPLAINATION.cohorts.values())

        exp_shap_abnormal = pd.DataFrame(
            cohort_exps[0].values, columns=cohort_exps[0].feature_names
        )  # .abs().mean()

        exp_shap_healty = pd.DataFrame(
            cohort_exps[1].values, columns=cohort_exps[1].feature_names
        )  # .abs().mean()


        feature_metrics = pd.concat(
            {
                "SHAP_healty": exp_shap_healty.abs().mean(),
                "SHAP_abnormal": exp_shap_abnormal.abs().mean(),
            },
            axis="columns",
        )

        # feature_metrics = (
        #     feature_metrics.join(internal_feature_metrics)
        #     .fillna(0)
        #     .sort_values(by="Gain", ascending=False)
        # )

        return feature_metrics
    
      
    
with open("params.yml", "r") as f:
        ext_params = yaml.load(f, Loader=yaml.FullLoader)
   
   
def objective(trial) -> float:
    """
    The function that runs a single model and evaluates it.
    """

    seed = trial.suggest_int("seed", 1, 10_000)

    # params={"objective":   "binary:logistic",
    #         "eval_metric": "logloss",
    #         'max_depth':   trial.suggest_int("max_depth", 2, 6, ),
    #         "eta":         trial.suggest_float("eta", 0.01, 0.3),
    #         "subsample":   trial.suggest_float("subsample", 0.5, 0.9),
    #         "lambda": trial.suggest_float("lambda", 0, 1),
    #         "alpha": trial.suggest_float("alpha",0,1),
    #         "scale_pos_weight": trial.suggest_float("scale_pos_weight",0,2)
    #     }
    
    params = {
            "objective": "binary:logistic",
            "eval_metric":       "logloss",
            "max_depth":                 5,
            "eta":                0.226908,
            "subsample":          0.749761,
            "lambda":             0.418621,
            "alpha":              0.466643,
            "scale_pos_weight":   1.640296,
    }
                
    model_instance = generate_model("HOMA-IR alterado", "data.csv", removed_features = ext_params["feature_engineering"]["removed_features"], xg_params = params, kfold_splits = 5, seed=seed)   
    
    return model_instance.get_AUC_on_test_data(), model_instance.get_feature_metrics()["SHAP_abnormal"]["fenilalax"]



In [162]:
import optuna
study_name: str = "alejandro_5"

study = optuna.create_study(
    study_name=study_name,
    storage="sqlite:///homa_studies.db",
    directions=["maximize", "maximize"],
    #sampler=optuna.samplers.TPESampler(), # NSGAIISampler(),
    sampler=optuna.samplers.NSGAIISampler(), # NSGAIISampler(),
    load_if_exists=True,
)

#study.optimize(objective, n_trials=1_000, n_jobs=-1)
# Sampling may take a long time. Other samples are faster but tend to be hiperfixed and thus generate repeated results. 


[I 2023-05-03 09:31:12,211] Using an existing study with name 'alejandro_5' instead of creating a new one.
[I 2023-05-03 09:31:14,846] Trial 733 finished with values: [1.0, 0.0] and parameters: {'seed': 5831}. 
[I 2023-05-03 09:31:15,077] Trial 732 finished with values: [0.5555555555555556, 0.0] and parameters: {'seed': 4005}. 
[I 2023-05-03 09:31:15,148] Trial 746 finished with values: [0.8055555555555555, 0.0] and parameters: {'seed': 3578}. 
[I 2023-05-03 09:31:15,149] Trial 734 finished with values: [0.6944444444444444, 0.0] and parameters: {'seed': 603}. 
[I 2023-05-03 09:31:15,149] Trial 738 finished with values: [0.8076923076923077, 0.0] and parameters: {'seed': 680}. 
[I 2023-05-03 09:31:15,149] Trial 731 finished with values: [0.7692307692307693, 0.2993854880332947] and parameters: {'seed': 2691}. 
[I 2023-05-03 09:31:15,295] Trial 737 finished with values: [0.9722222222222223, 0.10452819615602493] and parameters: {'seed': 4006}. 
[I 2023-05-03 09:31:15,295] Trial 741 finished

In [230]:
import pandas as pd
from optuna.visualization._pareto_front import (
    _get_pareto_front_info,
    _make_scatter_object,
    _make_marker,
    _make_hovertext,
)
from typing import Sequence
from optuna.trial import FrozenTrial
from optuna.visualization._plotly_imports import go


df_best    = trials_df(info.best_trials_with_values, "best")
df_nonbest = trials_df(info.non_best_trials_with_values, "nonbest")
# df         = pd.concat([df_best, df_nonbest])

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=df_nonbest["x"],
        y=df_nonbest["y"],
        mode="markers",
        marker=dict(
            size=8, opacity=0.5, symbol="circle", line=dict(width=1, color="black")
        ),
        name="Suboptimal",
    )
)
fig.add_trace(
    go.Scatter(
        x=df_best["x"],
        y=df_best["y"],
        mode="markers",
        marker=dict(
            size=8, opacity=0.5, symbol="square", line=dict(width=1, color="black")
        ),
        name="Pareto frontier",
    )
)
# Define the width and height of the plot
plot_width = 900
plot_height = 400
axis_label_font_size = 19

# Customize the plot
fig.update_layout(
    xaxis=dict(
        title=dict(text="Model performance (AUC)", font=dict(size=axis_label_font_size)), 
        range=[-0.05, 1.05]
    ),
    yaxis=dict(
        title=dict(text="Phe importance (Shapley value)", font=dict(size=axis_label_font_size)), 
        range=[-0.05, 0.85]
        )
    ,
    font=dict(size=18),
    # plot_bgcolor='white',
    legend=dict(x=0.3, y=1.1, bgcolor="rgba(255, 255, 255, 0)", orientation="h"),
    width=plot_width,
    height=plot_height,
    margin=dict(l=0, r=0, t=0, b=0),
)
x_max = max(df_nonbest["x"].max(), df_best["x"].max())

fig.update_layout(
    # ... (previous settings)
    shapes=[
        dict(
            type="rect",
            xref="x",
            yref="paper",
            x0=0.8,
            x1=x_max,
            y0=0.0,
            y1=1,
            fillcolor="darkgreen",
            opacity=0.20,
            layer="below",
            line_width=0,
        )
    ],
    # ... (previous settings)
)


import plotly.io as pio

dpi = 300  # Set the desired resolution (dots per inch)
output_filename = f"pareto_{study_name}.png"
pio.write_image(fig, output_filename, format="png", scale=dpi / 96)

# Show the plot
fig.show()

In [164]:
# # Extract the best trials from the multi-objective study
# best_trials = study.best_trials
# 
# # Print information about the best trials
# for i, trial in enumerate(best_trials):
#     print(f"Best trial {i+1} number: {trial.number}")
#     print(f"Best trial {i+1} values: {trial.values}")
#     print(f"Best trial {i+1} parameters: {trial.params}")
#     print()


# Ranking plots

In [204]:
# Create a empty data frame for 
bs_trials = pd.DataFrame(columns=[
 'alpha',
 'eta',
 'lambda',
 'max_depth',
 'scale_pos_weight',
 'seed',
 'subsample',
 'auc', 'phe'
])

hardcoded_params = {
            "objective": "binary:logistic",
            "eval_metric":       "logloss",
            "max_depth":                 5,
            "eta":                0.226908,
            "subsample":          0.749761,
            "lambda":             0.418621,
            "alpha":              0.466643,
            "scale_pos_weight":   1.640296,
    }

CUTOFF_AUC = 0.80
CUTOFF_PHE = 0.20

for t in study.get_trials(): #[:499]:
    if t.values[0] > CUTOFF_AUC and t.values[1] > CUTOFF_PHE:
        #bs_trials.loc[t.number] = list(t.params.values()) + list(t.values)
        bs_trials.loc[t.number] = [0.466643, 0.226908, 0.418621, 5, 1.640296] + list(t.params.values()) + [0.749761] + list(t.values)

# for t in study.best_trials:
# for t in better_trials:
#     bs_trials.loc[t.number] = list(t.params.values()) + list(t.values)
#     # and saves them to rows in a dataframe

bs_trials.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34 entries, 3 to 1730
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   alpha             34 non-null     float64
 1   eta               34 non-null     float64
 2   lambda            34 non-null     float64
 3   max_depth         34 non-null     float64
 4   scale_pos_weight  34 non-null     float64
 5   seed              34 non-null     float64
 6   subsample         34 non-null     float64
 7   auc               34 non-null     float64
 8   phe               34 non-null     float64
dtypes: float64(9)
memory usage: 2.7 KB


In [205]:
bs_trials = bs_trials.drop_duplicates()
bs_trials.describe().loc['50%']

alpha                  0.466643
eta                    0.226908
lambda                 0.418621
max_depth              5.000000
scale_pos_weight       1.640296
seed                6225.000000
subsample              0.749761
auc                    0.916667
phe                    0.264807
Name: 50%, dtype: float64

In [206]:
import plotly.express as px
#fig = px.parallel_coordinates(bs_trials)

fig = px.parallel_coordinates(
      bs_trials, 
      color="auc",
      dimensions=[
            'seed',
            'alpha',
            'eta',
            'lambda',
            'max_depth',
            'scale_pos_weight',
            'subsample',
            "phe", 
            "auc"
      ],)
fig.show()

In [207]:
import plotly.express as px

fig_boxplot = px.box(
    bs_trials.drop("seed", axis="columns"), 
    points="all", 
    facet_col="variable", 
)

fig_boxplot.update_xaxes(matches=None)
fig_boxplot.update_yaxes(matches=None)
fig_boxplot.for_each_yaxis(lambda yaxis: yaxis.update(showticklabels=True))
fig_boxplot.show()

In [208]:
bs_trials['seed'], bs_trials['max_depth'] = bs_trials['seed'].astype('int'), bs_trials['max_depth'].astype('int')
bs_trials.head()


,alpha,eta,lambda,max_depth,scale_pos_weight,seed,subsample,auc,phe
3,0.466643,0.226908,0.418621,5,1.640296,9101,0.749761,0.805556,0.286625
177,0.466643,0.226908,0.418621,5,1.640296,9413,0.749761,0.833333,0.231811
215,0.466643,0.226908,0.418621,5,1.640296,1049,0.749761,0.923077,0.258726
239,0.466643,0.226908,0.418621,5,1.640296,7677,0.749761,0.916667,0.320730
292,0.466643,0.226908,0.418621,5,1.640296,5509,0.749761,0.888889,0.228020


In [209]:
params ={"objective":   "binary:logistic",
            "eval_metric": "logloss"}


params.update(
    bs_trials.drop(["auc","phe"], axis="columns").iloc[1].to_dict()
)

params

{'objective': 'binary:logistic',
 'eval_metric': 'logloss',
 'alpha': 0.466643,
 'eta': 0.226908,
 'lambda': 0.418621,
 'max_depth': 5.0,
 'scale_pos_weight': 1.640296,
 'seed': 9413.0,
 'subsample': 0.749761}

In [210]:
bs_trials

,alpha,eta,lambda,max_depth,scale_pos_weight,seed,subsample,auc,phe
3,0.466643,0.226908,0.418621,5,1.640296,9101,0.749761,0.805556,0.286625
177,0.466643,0.226908,0.418621,5,1.640296,9413,0.749761,0.833333,0.231811
215,0.466643,0.226908,0.418621,5,1.640296,1049,0.749761,0.923077,0.258726
239,0.466643,0.226908,0.418621,5,1.640296,7677,0.749761,0.916667,0.320730
292,0.466643,0.226908,0.418621,5,1.640296,5509,0.749761,0.888889,0.228020
341,0.466643,0.226908,0.418621,5,1.640296,6848,0.749761,0.846154,0.223445
356,0.466643,0.226908,0.418621,5,1.640296,1410,0.749761,0.944444,0.325203
394,0.466643,0.226908,0.418621,5,1.640296,7054,0.749761,0.916667,0.378276
442,0.466643,0.226908,0.418621,5,1.640296,2139,0.749761,0.807692,0.444731
478,0.466643,0.226908,0.418621,5,1.640296,84,0.749761,0.846154,0.630811


In [211]:

paretos = {}

df_ranks = pd.DataFrame()

for number in bs_trials.index: 


    instance_params = bs_trials.loc[number][['max_depth','eta','subsample','lambda','alpha','scale_pos_weight']].to_dict()
    instance_params['max_depth'] = int(instance_params['max_depth'])

    instance_params['objective'] = 'binary:logistic'
    instance_params['eval_metric'] = 'logloss'

    print(instance_params)

    model_instance = generate_model(
        "HOMA-IR alterado", "data.csv", 
        removed_features = ext_params["feature_engineering"]["removed_features"], 
        xg_params = instance_params,
        kfold_splits = 5, 
        seed = int(bs_trials.loc[number]['seed'])
    )   
# 
    print(
        model_instance.model.attributes()
    )
    
    df_ranks[number] = model_instance.get_feature_metrics()["SHAP_abnormal"]

    print(
        'Phe_value : ', model_instance.get_feature_metrics()["SHAP_abnormal"]["fenilalax"], "\t",
        'Phe_ranking : ',model_instance.get_feature_metrics()["SHAP_abnormal"].rank(ascending=False)["fenilalax"]
    )

    print('\n')





{'max_depth': 5, 'eta': 0.226908, 'subsample': 0.749761, 'lambda': 0.418621, 'alpha': 0.466643, 'scale_pos_weight': 1.640296, 'objective': 'binary:logistic', 'eval_metric': 'logloss'}
{'best_iteration': '5', 'best_ntree_limit': '6', 'best_score': '0.4837466801206271'}
Phe_value :  0.28662467 	 Phe_ranking :  2.0


{'max_depth': 5, 'eta': 0.226908, 'subsample': 0.749761, 'lambda': 0.418621, 'alpha': 0.466643, 'scale_pos_weight': 1.640296, 'objective': 'binary:logistic', 'eval_metric': 'logloss'}
{'best_iteration': '34', 'best_ntree_limit': '35', 'best_score': '0.27833866017560166'}
Phe_value :  0.23181069 	 Phe_ranking :  9.0


{'max_depth': 5, 'eta': 0.226908, 'subsample': 0.749761, 'lambda': 0.418621, 'alpha': 0.466643, 'scale_pos_weight': 1.640296, 'objective': 'binary:logistic', 'eval_metric': 'logloss'}
{'best_iteration': '23', 'best_ntree_limit': '24', 'best_score': '0.4155168191840251'}
Phe_value :  0.25872648 	 Phe_ranking :  7.0


{'max_depth': 5, 'eta': 0.226908, 'subsample': 

In [212]:
df_ranks_long = df_ranks.T
order = df_ranks_long.rank(axis="columns", ascending=False).median(axis="rows").sort_values().index

# Create a grouped barplot using Plotly
fig_barplot = px.bar(df_ranks, barmode='group', template='ggplot2')
fig_barplot.update_xaxes(categoryorder='array', categoryarray=order)

# Show the plot
fig_barplot.show()

# fig = px.bar(, x='variable', y='value')
# fig.show()

In [214]:
fig = px.parallel_coordinates( df_ranks.T )
      # color="auc",
      # dimensions=[
      #       'seed',
      #       'alpha',
      #       'eta',
      #       'lambda',
      #       'max_depth',
      #       'scale_pos_weight',
      #       'subsample',
      #       "phe", 
      #       "auc"
      # ])
fig.show()